In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import re
from google.cloud import bigquery
import plotly.graph_objects as go


from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_Online_spend_abroad = '''SELECT time_period_value, spend, merchant_channel
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value IN ('2024Q1','2024Q2','2024Q3','2024Q4')
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and destination_country != 'UNITED KINGDOM'
and mcg = 'All'
and mcc = 'All'
GROUP BY time_period_value, merchant_channel, spend
ORDER BY time_period_value, spend DESC'''

Online_ratio_abroad = bq.read_bq_table_sql(client, UK_Online_spend_abroad)

Online_ratio_abroad['year'] = Online_ratio_abroad['time_period_value'].str[:4]

Online_ratio_abroad.head()

In [ ]:
Online_ratio_abroad.to_csv('Online_ratio_abroad.csv')

In [ ]:
# Calculate total spend for Online channel
total_online_spend = Online_ratio_abroad[Online_ratio_abroad['merchant_channel'] == 'Online']['spend'].sum()

# Calculate total spend in all category
total_spend = Online_ratio_abroad[Online_ratio_abroad['merchant_channel'] == 'All']['spend'].sum()

# Calculate total spend in all category
total_Face_to_face_spend = Online_ratio_abroad[Online_ratio_abroad['merchant_channel'] == 'Face to Face']['spend'].sum()

# Calculate the online spend ratio for the entire year
online_spend_ratio_1 = total_online_spend / total_spend

# Calculate the online spend ratio for the entire year
online_spend_ratio_2 = total_online_spend / (total_online_spend+total_Face_to_face_spend)


# Display the result
print(online_spend_ratio_1)
print(online_spend_ratio_2)


